In [200]:
#Importação das bibliotecas necessárias
import graphviz
import pandas as pd

from sklearn import tree
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 

In [201]:
#Método que irá gerar métrica para classificação das casas
def house_class(n):
    if (n<-0.5):
        return 0
    elif (n<0.5):
        return 1
    else:
        return 2

In [202]:
#Leitura do csv com as informações das casas
data = pd.read_csv('kc_house_data.csv', sep=',')

#print(data)

In [203]:
#Filtragem dos dados pelo zipcode
filtered_data = data.query('zipcode==98008')
filtered_data = filtered_data.reset_index(drop=True)

#print(filtered_data)

In [204]:
#criando coluna com preço/metro²
filtered_data ['price_sqft'] = filtered_data['price']/filtered_data['sqft_living']

#print(filtered_data)

In [205]:
#cálculo para zscore
avg_prc = filtered_data['price_sqft'].mean()
#print(avg_prc)
std_prc = filtered_data['price_sqft'].std()
#print(std_prc)

zscore = (filtered_data['price_sqft'] - avg_prc) / std_prc

#adicionando coluna do zscore
filtered_data['zscore'] = zscore

#print(filtered_data)

In [206]:
#aplicando o zscore para criação da coluna de avaliação
filtered_data['pricing'] = filtered_data['zscore'].apply(house_class)

#print(filtered_data)

#salva arquivo csv filtrado e tratado
filtered_data.to_csv(r'filtered_data.csv', index=False)

#lista dos 10 ultimos valores
aval_data = filtered_data.tail(10)

#print(aval_data)

#salva arquivo csv
aval_data.to_csv(r'aval_data.csv', index=False)

In [207]:
#criação da árvore
dec_tree = tree.DecisionTreeClassifier(min_samples_leaf=10, min_impurity_decrease=0.001, min_samples_split=20)
x = filtered_data[['bedrooms', 'bathrooms', 'floors', 'view', 'condition']]
y = filtered_data['pricing']
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)
dec_tree.fit(x_train, y_train)


with open("decision_tree.dot", 'w') as f:
    f = tree.export_graphviz(dec_tree, out_file=f, feature_names=x.columns, rounded=True, filled=True, special_characters=True)
print('.dot gerado')

.dot gerado


In [208]:
#Teste de accuracy
predictions=dec_tree.predict(x_test)
print(accuracy_score(y_test,predictions))

0.543859649122807


In [209]:
#Comparação das 10 últimas casas da lista

predict = list(dec_tree.predict(x[-10:]))
print(predict)

ten_last = list(filtered_data['pricing'][-10:])
print(ten_last)

[0, 1, 1, 1, 1, 2, 0, 2, 1, 2]
[0, 2, 1, 1, 0, 2, 2, 2, 1, 2]
